In [ ]:
import pandas as pd

dir = "common-sense"

test_file = f"{dir}/test_data.csv"
df_test = pd.read_csv(test_file)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMultipleChoice.from_pretrained("results/bert-base-uncased/checkpoint-500")

In [ ]:
options = ["OptionA", "OptionB", "OptionC"]
x = ['A', 'B', 'C']

def predict(example):
    prompts = []
    for opt in options:
        prompts.append([example['FalseSent'], example[opt]])

    inputs = tokenizer(prompts, return_tensors="pt", padding=True)
    labels = torch.tensor(0).unsqueeze(0)
    
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    return x[predicted_class]

df_test['Label'] = df_test.apply(lambda row: predict(row), axis=1)

In [ ]:
df_test.head()

In [ ]:

submission_df = df_test[["id", "Label"]].copy()
submission_df = submission_df.rename(columns={"Label": "answer"})
submission_df.set_index("id", inplace=True)
submission_df.to_csv(f"submissions/Submission_10-12-2022.csv")